In [3]:
import audiosegment

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np
np.set_printoptions(suppress=True)
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt

In [5]:
SAMPLE_RATE = 32000
SAMPLE_WIDTH = 1./SAMPLE_RATE
FRAME_DURATION = 4
FRAME_WIDTH = int(SAMPLE_RATE * FRAME_DURATION)
MIN_FREQ = 1./FRAME_DURATION

def FREQ_BY_NUM(i):
    return MIN_FREQ * i

In [6]:
def FFT(frame):
    fft = np.abs(np.fft.rfft(frame).real)
    return fft

def CUTFFT(fft, max_freq):
    return fft[:int(max_freq*FRAME_DURATION)]

def SIGNAL_MEAN(signal):
    return np.mean(signal)

def SIGNAL_GEOM_MEAN(signal):
    return np.exp(np.sum(np.log(1e-11 + signal))/len(signal))

def FFT_MAX_FREQ(fft):
    return FREQ_BY_NUM(np.argmax(fft))

def SFM(fft):
    return 10 * np.log10(1e-11 + SIGNAL_GEOM_MEAN(fft)/(1e-11 + SIGNAL_MEAN(fft)))
    
def ENERGY(signal):
    return SAMPLE_WIDTH*np.sum(signal.real**2) #

def IDENTITY(fft):
    return fft

In [7]:
import pyaudio
import time
import wave

p=pyaudio.PyAudio()
stream=p.open(format=pyaudio.paInt16, channels=1, rate=SAMPLE_RATE, input=True, frames_per_buffer=FRAME_WIDTH)

DURATION = 10
PLOT_EVERY = 1


wavefile = wave.open("tmp.wav", "wb")
wavefile.setnchannels(1)
wavefile.setsampwidth(p.get_sample_size(pyaudio.paInt16))
wavefile.setframerate(SAMPLE_RATE)


for i in range(int(DURATION/FRAME_DURATION)):
    current_time = i * FRAME_DURATION
    rawdata = stream.read(FRAME_WIDTH)
    data_int = np.frombuffer(rawdata, dtype=np.int16)
    data = np.array(data_int, dtype='float32')
    seg = audiosegment.from_numpy_array(data_int, framerate=SAMPLE_RATE)
    results = seg.detect_voice()
    voiced = [tup[1] for tup in results if tup[0] == 'v']
    unvoiced = [tup[1] for tup in results if tup[0] == 'u']
    print(voiced)
    wavefile.writeframes(rawdata)
    voiced_segment = voiced[0].reduce(voiced[1:])
    voiced_segment.export(f"{i}voiced.wav", format="WAV")
    print("Reducing unvoiced segments to a single wav file 'unvoiced.wav'")
    unvoiced_segment = unvoiced[0].reduce(unvoiced[1:])
    unvoiced_segment.export(f"{i}unvoiced.wav", format="WAV")
    
    
stream.stop_stream()
stream.close()
p.terminate()

[: 1 channels, 16 bit, sampled @ 32.0 kHz, 0.240s long, : 1 channels, 16 bit, sampled @ 32.0 kHz, 0.240s long, : 1 channels, 16 bit, sampled @ 32.0 kHz, 0.060s long]
Reducing unvoiced segments to a single wav file 'unvoiced.wav'


KeyboardInterrupt: 